In [63]:
# M2PY -- Python library used to control the Makergear M2 Pneumatic Control System [M2PCS]
# Developed in the Architected Materials Laboratory at the University of Pennsylvania
# Author: Matthew Sorna [sorna@seas.upenn.edu]

# Importing of necessary dependent modules
import os
import serial
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Module Function Definitions

# Serial communication commands
def mopen(com, baud, printout = 1, fid = os.path.abspath('path_vis_temp.txt')):
    """
    Returns the necessary handle (serial object if printout = 1, or temporary file if printout = 0). If printout = 1, this function will instantiate a serial object used by all subsequent function calls to send serial commands to the specified printer. If printout = 0, this function will store all revelant coordinate changes (move and arc commands) to a temporary file that can then be used to visualize print paths before sending commands to the printer. By default, printout = 1.
    """
    global pflag
    if printout == 1:
        pflag = 1
        handle = serial.Serial(com, baud, timeout = 1)
        time.sleep(2) # Make sure to give it enough time to initialize
        print('Serial port connected')
        for _ in range(21): # Reads in all 21 lines of initialization text for the M2
            handle.readline()
        return handle
        
    elif printout == 0:
        global print_dir
        global coord_sys
        print_dir = fid 
        coord_sys = 'abs'
        
        pflag = 0
        handle = open(fid, "w")
        return handle
       
def mclose(handle):
    """
    Closes the specified handle. If in mopen printout = 1, this function will close the necessary serial object. If prinout = 0, this function will close the specified temporary file and plot a visualization of all relevant movement commands. Visualization function will use whatever coordinate system you explicity designate using coord. If coord isn't explicitly called, the coordinate system used by the visualization tool will be absolute.
    """
    global pflag
    if pflag == 1:
        print("Serial port disconnected")
        handle.close()
        
    elif pflag == 0:
        global print_dir
        global coord_sys
        handle.close()
        path_vis(fid = print_dir, coord = coord_sys)

def path_vis(fid, coord = 'abs'):
    """
    Takes the (x, y, z) coordinates generated from mp.mopen(printout = 0), and plots them into a 3D line graph to check a print path before actually sending commands to the Makergear. Visualization function will use whatever coordinate system you explicity designate using coord. If coord isn't explicitly called, the coordinate system used by the visualization tool will be absolute. When using path_vis, the file directory of the path coordinates needs to be explicity set, unlike when it is implictly called inside mclose.
    """
    coord_array = np.zeros([1, 3])
    
    if coord == 'abs':
        with open(fid, "r") as data:
            for line in data:
                raw = line.split(' ')
                coords = [float(raw[0]), float(raw[1]), float(raw[2])]
                coord_array = np.append(coord_array, [coords[0], coords[1], coords[2]])
        coord_array.shape = (int(len(coord_array)/3), 3)
        
    elif coord == 'rel':
        old_coords = [0,0,0]
        with open(fid, "r") as data:
            for line in data:
                raw = line.split(' ')
                coords = [float(raw[0]), float(raw[1]), float(raw[2])]
                coord_array = np.append(coord_array, [coords[0] + old_coords[0], coords[1] + old_coords[1], coords[2] + old_coords[2]])
                old_coords =  [coords[0] + old_coords[0], coords[1] + old_coords[1], coords[2] + old_coords[2]]
        coord_array.shape = (int(len(coord_array)/3), 3)
    
    x_coord = coord_array[:,0]
    y_coord = coord_array[:,1]
    z_coord = coord_array[:,2]
    
    xmin = np.min(x_coord)
    xmax = np.max(x_coord)
    ymin = np.min(y_coord)
    ymax = np.max(y_coord)
    
    fig = plt.figure()
    ax = fig.gca(projection=Axes3D.name)
    ax.set_xlim3d(xmin, xmax)
    ax.set_ylim3d(ymin, ymax)
    ax.set_zlim3d(0, 203)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.plot(x_coord, y_coord, z_coord, color = 'g', linewidth = 1.0, label = 'Print path visualization')
    ax.legend()
    plt.show()           
        
def prompt(com, baud):
    """
    Allows for quick, native GCode serial communication with the M2, provided that the proper com port and baud rate are selected, and match what is found in system settings. To exit the command prompt environment, just type exit in the IPython console.
    """
    escape = 0
    cmd = ''
    
    #Create and define serial port parameters
    handle = serial.Serial(com, baud, timeout = 0)
    time.sleep(2) # Make sure to give it enough time to initialize
    print("Enter a GCode command. To exit, type \'exit\'")
    
    handle.write(str.encode('M17\n'))
    
    x = 0
    y = 0
    z = 0
    
    xval = 0
    yval = 0
    zval = 0
    
    while escape == 0:
        cmd = input('>> ')
        if cmd == 'exit':
            handle.close()
            print("Serial port disconnected")
            escape = 1
        else:
            handle.write(str.encode('{}\n'.format(cmd)))
            
            if cmd =='G28':
                x = 0
                y = 0
                z = 0
                
                xval = 0
                yval = 0
                zval = 0
            
            cmd_split = cmd.split(' ')
            
            for a in cmd_split:
                xchar = a.split('X')
                ychar = a.split('Y')
                zchar = a.split('Z')
                if xchar[0] == '' and len(xchar) > 1 and xchar[1] != '':
                    xval = float(xchar[1])
                    x = x + xval
                if ychar[0] == '' and len(ychar) > 1 and ychar[1] != '':
                    yval = float(ychar[1])
                    y = y + yval
                if zchar[0] == '' and len(zchar) > 1 and zchar[1] != '':
                    zval = float(zchar[1])
                    z = z + zval
            print('Currently at ({}, {}, {})'.format(x, y, z))

def file_read(fid, com, baud):
    """
    Reads in a text file of GCode line by line, and waits for the M2 to acknowledge that it received the command before sending another, maintaining print accuracy.
    """
    #Create and define serial port parameters
    handle = serial.Serial(com, baud, timeout = 100)
    time.sleep(2) # Make sure to give it enough time to initialize
    
    print('Serial port initialized')
    for _ in range(21): # Reads in all 21 lines of initalization text for the M2
        handle.readline()
    print('Beginning print')
    with open(fid, "r") as gcode:
        for line in gcode:
            split_line = line.split(';') # chops off comments
            if split_line[0] != '':      # makes sure it's not a comment-only line of GCode
                handle.write(str.encode('{}\n'.format(split_line[0]))) #Reads in text file (GCode) line by line and sends commands to M2
                print(split_line[0])
                read = handle.readline()
                while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = handle.readline()
                
    print('Print complete!\nSerial port closed')
    handle.close() #Closes serial port

# GCode wrappers

# G0/G1       
def move(handle, x = 0, y = 0, z = 0, track = 1):
    """
    Moves to the specified point, keeping in mind the coordinate system (relative / absolute)
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('G1 X{} Y{} Z{}\n'.format(x, y, z)))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Move to ({}, {}, {})'.format(x, y, z))
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
            
    elif pflag == 0 and track == 1:
        handle.write('{} {} {}\n'.format(x, y, z))


def speed(handle, speed = 30):
    """
    Sets the movement speed of the printer to the specified speed in [mm/s] (default 30 mm/sec)
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('G1 F{}\n'.format(speed*60)))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Changing movement speed to {} mm/s'.format(speed))

def rotate(handle, speed = 0):
    """
    Sets the rotation speed of the printer to the specified speed
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M9 S{}\n'.format(speed)))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Changing rotation speed to {} mm/s'.format(speed))
		    
# G2/G3
def arc(handle, x = 0, y = 0, z = 0, i = 0, j = 0, direction = 'ccw'):
    """
    Moves to the specified x-y point, with the i-j point as the center of the arc, with direction specified as 'cw' or 'ccw' (default 'ccw')
    """
    global pflag
    if pflag == 1:
        if direction == 'ccw':
            handle.write(str.encode('G3 X{} Y{} Z{} I{} J{}\n'.format(x, y, z, i, j)))
            read = handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = handle.readline()
                time.sleep(0.06)
            print('CCW arc to ({}, {}, {}), center at ({}, {})'.format(x, y, z, i, j))
        elif direction == 'cw':
            handle.write(str.encode('G2 X{} Y{} Z{} I{} J{}\n'.format(x, y, z, i, j)))
            read = handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = handle.readline()
                time.sleep(0.06)
            print('CW arc to ({}, {}, {}), center at ({}, {})'.format(x, y, z, i, j))
    
    elif pflag == 0:
        handle.write('{} {} {}\n'.format(x, y, z))
        
# G4
def wait(handle, seconds = 0):
    """
    Waits for the specified amount of time (default 0 seconds)
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('G4 S{}\n'.format(seconds)))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = handle.readline()
                time.sleep(0.06)
        print('Waiting for {} seconds'.format(seconds))

# G28
def home(handle, axes = 'X Y Z'):
    """
    Homes the specified axes (default 'X Y Z')
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('G28 {}\n'.format(axes)))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = handle.readline()
                time.sleep(0.06)
        print('{} axes homed'.format(axes))
        
# G90/G91
def coord(handle, coord = 'abs'):
    """
    Sets the coordinate system of the printer [relative or absolute] (default 'abs')
    """
    global coord_sys
    coord_sys = coord
    
    global pflag
    if pflag == 1:
        if coord == 'abs':
            handle.write(str.encode('G90\n'))
            read = handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = handle.readline()
                time.sleep(0.06)
            print('Set to absolute coordinates')
            
        elif coord == 'rel':
            handle.write(str.encode('G91\n'))
            read = handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = handle.readline()
                time.sleep(0.06)
            print('Set to relative coordinates')  

# G92       
def set_coords(handle, x = 0, y = 0, z = 0):
    """
    Sets the current position to the specified (x, y, z) point (keeping in mind the current coordinate system)
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('G92 X{} Y{} Z{}\n'.format(x, y, z)))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Changed current position to ({}, {}, {})'.format(x, y, z))        

# CORE SHELL SPECIFIC FUNCTIONS
def allon(handle):
    """
    Turns pneumatic CHANNEL 1, 2, 3 ON
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M3\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        handle.write(str.encode('M5\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        handle.write(str.encode('M7\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('All Channels ON')

def alloff(handle):
    """
    Turns pneumatic CHANNEL 1, 2, 3 OFF
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M4\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        handle.write(str.encode('M6\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        handle.write(str.encode('M8\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('All Channels OFF') 

# M3    
def ch1on(handle):
    """
    Turns pneumatic CHANNEL 1 ON
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M3\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Channel 1 ON')

# M4
def ch1off(handle):
    """
    Turns pneumatic CHANNEL 1 OFF
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M4\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Channel 1 OFF')

# M5    
def ch2on(handle):
    """
    Turns pneumatic CHANNEL 2 ON
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M5\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Channel 2 ON')

# M6
def ch2off(handle):
    """
    Turns pneumatic CHANNEL 2 OFF
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M6\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Channel 2 OFF')

# M7
def ch3on(handle):
    """
    Turns pneumatic CHANNEL 3 ON
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M7\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Channel 3 ON')

# M8
def ch3off(handle):
    """
    Turns pneumatic CHANNEL 3 OFF
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M8\n'))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Channel 3 OFF')
    
def delay_set(handle, delay = 50):
    """
    Sets the delay time (in ms) between a channel turning on and the execution of another command. Can be used to fine tune under extrusion effects, depending on ink viscosity.
    """
    global pflag
    if pflag == 1:
        handle.write(str.encode('M50 S{}\n'.format(delay)))
        read = handle.readline()
        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
            read = handle.readline()
            time.sleep(0.06)
        print('Channel ON delay time changed to {} ms'.format(delay))

def clip(handle, clip_height = 1, radius = 0.5):
    """
    This subroutine automatically turns off all channels, and performs a quick arc/z-translation to shear excess material away from nozzle before continuing with print path.
    """
    global pflag
    if pflag == 1:
        alloff(handle)
        arc(handle, z = clip_height, i = radius)
        print('Print clipped')

def change_tool(handle, dx = 0, dy = 0, change_height = 10):
    """
    This subroutine automatically turns off all channels, and performs a predetermined z translation of z = change_height, and then moves (x,y) = (dx, dy) to allow for change between multiple nozzles. It also automatically lowers back to the z height it was at previously, continuing printing after switching active tools
    """
    global pflag
    if pflag == 1:
        alloff(handle)
        move(handle, z = change_height)
        move(handle, x = dx, y = dy)
        move(handle, z = -change_height)
        print('Tool change to ({}, {})'.format(dx, dy))

In [64]:
# m2py printer functions


# M2PY -- Python library used to control the Makergear M2 Pneumatic Control System [M2PCS]
# Developed in the Architected Materials Laboratory at the University of Pennsylvania
# Author: Matthew Sorna [sorna@seas.upenn.edu]

# Importing of necessary dependent modules
import os
import serial
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches

# Module Function Definitions

class Makergear:
    def __init__(self, com, baud, printout = 0, verbose = True):
        self.com = com
        self.baud = baud
        self.printout = printout
        self.verbose = verbose
        self.fid = os.path.abspath('path_vis_temp.txt')
        self.channel_status = np.array([0,0,0])
        self.coords = np.array([0,0,0])
        self.current_tool = 1
        self.tool_coords = np.array([[0,0,0],[0,0,0],[0,0,0]])

        if self.printout == 1:
            self.handle = serial.Serial(com, baud, timeout = 1)
            time.sleep(2) # Make sure to give it enough time to initialize
            if self.verbose: print('Connecting to {}'.format(self.com))
            for _ in range(21): # Reads in all 21 lines of initialization text for the M2
                self.handle.readline()
        elif self.printout == 0:
            self.current_coord_sys = 'abs'
            self.handle = open(self.fid, "w")

    def close(self, zrange = [0, 203]):
        """
        Closes the specified handle. If self.printout = 1, this function will close the necessary serial object. If prinout = 0, this function will close the specified temporary file and plot a visualization of all relevant movement commands. Visualization function will use whatever coordinate system you explicity designate using coord. If coord isn't explicitly called, the coordinate system used by the visualization tool will be absolute.
        """
        if self.printout == 1:
            self.alloff()
            self.rotate(speed = 0)
            if self.verbose: print('Disconnecting from {}'.format(self.com))
            self.handle.close()
        elif self.printout == 0:
            self.handle.close()
            self.path_vis(zrange)
            os.remove(self.fid)

    # GCode wrappers
    # G0/G1
    def move(self, x = 0, y = 0, z = 0, track = 1):
        """
        Moves to the specified point, keeping in mind the coordinate system (relative / absolute)
        """
        try:
            if self.current_coord_sys == 'abs':
                self.coords = np.array([x, y, z])
            elif self.current_coord_sys == 'rel':
                self.coords = self.coords + np.array([x, y, z])

            if self.printout == 1:
                if self.verbose: print('Moving to ({}, {}, {})'.format(x, y, z))
                self.handle.write(str.encode('G1 X{} Y{} Z{}\n'.format(x, y, z)))
                read = self.handle.readline()
                while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = self.handle.readline()
                    time.sleep(0.06)
                while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = self.handle.readline()
                    time.sleep(0.06)

            elif self.printout == 0 and track == 1:
                self.handle.write('{} {} {} {} {} {}\n'.format(x, y, z, self.channel_status[0], self.channel_status[1], self.channel_status[2]))
        except:
            self.handle.write(str.encode('M112\n'))
            self.close()
            raise ValueError('Emergency Stop! Turning off channels and disconnecting from {}'.format(self.com))

    # G2/G3
    def arc(self, x = 0, y = 0, i = 0, j = 0, direction = 'ccw'):
        r = np.sqrt((x - i)**2 + (y - j)**2)
        start_ang = np.angle(i*-1 + j*-1j)
        stop_ang = np.angle((x-i)*1 + (y-j)*1j)

        if start_ang < 0:
            start_ang = start_ang + 2*np.pi
            if stop_ang > 0:
                stop_ang = stop_ang + 2*np.pi

        if stop_ang < 0:
            stop_ang = stop_ang + 2*np.pi

        if direction == 'ccw':
            if stop_ang > start_ang:
                dtheta = (stop_ang - start_ang)
                s = int(np.ceil((r*dtheta)))
                theta = np.linspace(start_ang, stop_ang, s)
            elif start_ang > stop_ang:
                stop_ang = stop_ang + 2*np.pi
                dtheta = (stop_ang - start_ang)
                s = int(np.ceil((r*dtheta)))
                theta = np.linspace(start_ang, stop_ang, s)
            elif start_ang == stop_ang:
                stop_ang = stop_ang + 2*np.pi
                dtheta = (stop_ang - start_ang)
                s = int(np.ceil((r*dtheta)))
                theta = np.linspace(start_ang, stop_ang, s)

        elif direction == 'cw':
            if stop_ang > start_ang:
                dtheta = 2*np.pi - (stop_ang - start_ang)
                s = int(np.ceil((r*dtheta)))
                theta = np.flip(np.linspace(stop_ang, start_ang + 2*np.pi, s), axis = 0)
            elif start_ang > stop_ang:
                dtheta = (start_ang - stop_ang)
                s = int(np.ceil((r*dtheta)))
                theta = np.flip(np.linspace(stop_ang, start_ang, s), axis = 0)
            elif start_ang == stop_ang:
                stop_ang = stop_ang + 2*np.pi
                dtheta = (stop_ang - start_ang)
                s = int(np.ceil((r*dtheta)))
                theta = np.flip(np.linspace(start_ang, stop_ang, s), axis = 0)

        xpts = r*np.cos(theta)
        ypts = r*np.sin(theta)

        dxpts = np.array([])
        dypts = np.array([])

        for ii in range(s - 1):
            dxpts = np.append(dxpts, xpts[ii+1] - xpts[ii])
            dypts = np.append(dypts, ypts[ii+1] - ypts[ii])

        if self.current_coord_sys == 'abs':
            current_coords = self.return_current_coords()
            dxpts = dxpts + current_coords[0]
            dypts = dypts + current_coords[1]
            zpt = current_coords[2]
            if self.printout == 1:
                for jj in range(s-1):
                    old_verbose = self.verbose
                    self.verbose = False
                    self.move(x = dxpts[jj], y = dypts[jj], z = zpt)
                    self.verbose = old_verbose
                if self.verbose: print('Moving in a {} arc to ({},{}) with center ({},{})'.format(direction, x,y,i,j))
            elif self.printout == 0:
                for jj in range(s-1):
                    self.handle.write('{} {} {} {} {} {}\n'.format(dxpts[jj], dypts[jj], zpt, self.channel_status[0], self.channel_status[1], self.channel_status[2]))

        elif self.current_coord_sys == 'rel':
            if self.printout == 1:
                if self.verbose: print('Moving in a {} arc to ({},{}) with center ({},{})'.format(direction, x,y,i,j))
                for jj in range(s-1):
                    old_verbose = self.verbose
                    self.verbose = False
                    self.move(x = dxpts[jj], y = dypts[jj], z = 0)
                    self.verbose = old_verbose
            elif self.printout == 0:
                for jj in range(s-1):
                    self.handle.write('{} {} {} {} {} {}\n'.format(dxpts[jj], dypts[jj], 0, self.channel_status[0], self.channel_status[1], self.channel_status[2]))

    def speed(self, speed = 0):
        """
        Sets the movement speed of the printer to the specified speed in [mm/s] (default 0 mm/sec)
        """
        if self.printout == 1:
            if self.verbose: print('Setting movement speed to {} mm/s'.format(speed))
            self.handle.write(str.encode('G1 F{}\n'.format(speed*60)))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)

    def rotate(self, speed = 0):
            """
            Sets the rotation speed of the motor to the specified speed [0-127] (default 0)
            """
            if self.printout == 1:
                if self.verbose: print('Setting rotation speed to {}'.format(int(speed)))
                self.handle.write(str.encode('M9 S{}\n'.format(int(speed))))
                read = self.handle.readline()
                while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = self.handle.readline()
                    time.sleep(0.06)

    def ramp(self, start = 0, stop = 0, seconds = 1):
            """
            Sets the rotation speed of the motor to the specified speed [0-127] (default 0 --> 0)
            """
            if self.printout == 1:
                if self.verbose: print('Changing rotation from {} to {} in {} seconds'.format(int(start),int(stop), seconds))
                diff = stop - start
                steps = abs(stop - start)
                if steps > 0:
                    for i in range(steps):
                        if diff > 0:
                            dt = seconds / steps
                            self.handle.write(str.encode('M9 S{}\n'.format(int(start + i))))
                            read = self.handle.readline()
                            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                                read = self.handle.readline()
                                time.sleep(0.06)
                        elif diff < 0:
                            dt = seconds / steps
                            self.handle.write(str.encode('M9 S{}\n'.format(int(start - i))))
                            read = self.handle.readline()
                            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                                read = self.handle.readline()
                                time.sleep(0.06)


                        self.handle.write(str.encode('G4 S{}\n'.format(dt)))
                        read = self.handle.readline()
                        while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                                read = self.handle.readline()
                                time.sleep(0.06)

    # G4
    def wait(self, seconds = 0):
        """
        Waits for the specified amount of time (default 0 seconds)
        """
        if self.printout == 1:
            if self.verbose: print('Waiting for {} seconds'.format(seconds))
            self.handle.write(str.encode('G4 S{}\n'.format(seconds)))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = self.handle.readline()
                    time.sleep(0.06)

    # G28
    def home(self, axes = 'X Y Z'):
        """
        Homes the specified axes (default 'X Y Z')
        """
        split_axes = axes.split(' ')
        for i in range(len(split_axes)):
            if split_axes[i] == 'X':
                self.coords[0] = 0
            elif split_axes[i] == 'Y':
                self.coords[1] = 0
            elif split_axes[i] == 'Z':
                self.coords[2] = 0

        if self.printout == 1:
            if self.verbose: print('Homing {} axes'.format(axes))
            self.handle.write(str.encode('G28 {}\n'.format(axes)))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = self.handle.readline()
                    time.sleep(0.06)

    # G90/G91
    def coord_sys(self, coord_sys = 'abs'):
        """
        Sets the coordinate system of the printer [relative or absolute] (default 'abs')
        """
        self.current_coord_sys = coord_sys

        if self.printout == 1:
            if self.current_coord_sys == 'abs':
                if self.verbose: print('Setting to absolute coordinates')
                self.handle.write(str.encode('G90\n'))
                read = self.handle.readline()
                while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = self.handle.readline()
                    time.sleep(0.06)

            elif self.current_coord_sys == 'rel':
                if self.verbose: print('Setting to relative coordinates')
                self.handle.write(str.encode('G91\n'))
                read = self.handle.readline()
                while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = self.handle.readline()
                    time.sleep(0.06)

    # G92
    def set_current_coords(self, x = 0, y = 0, z = 0):
        """
        Sets the current position to the specified (x, y, z) point (keeping in mind the current coordinate system)
        """
        old_coords = self.coords
        self.coords = [x, y, z]

        if self.printout == 1:
            if self.verbose: print('Changing current position at ({}, {}, {}) to ({}, {}, {})'.format(old_coords[0], old_coords[1], old_coords[2], x, y, z))
            self.handle.write(str.encode('G92 X{} Y{} Z{}\n'.format(x, y, z)))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)

    def return_current_coords(self):
        """
        Returns the current stored coordinates of the Makergear object
        """
        if self.verbose: print('Current position: ({}, {}, {})'.format(self.coords[0], self.coords[1], self.coords[2]))
        return self.coords

    def set_bed_temp(self, temp = 25, wait = 'off'):
        """
        Sets the temperature of the heated bed to the specified temp. If the wait argument is set to 'on', the printer will wait for temp to be reached before excecuting other commands. If 'off' the printer will set the temp without waiting.
        """
        if temp > 100: 
            self.close()
            raise ValueError('Attempting to set heated bed to temperature above upper limit [100C]')
        
        if self.printout == 1:
            if wait == 'off':
                self.handle.write(str.encode('M140 S{}\n'.format(temp)))
                if self.verbose: print('Setting bed temp to {}C'.format(temp))
            elif wait == 'on':
                self.handle.write(str.encode('M190 S{}\n'.format(temp)))
                if self.verbose: print('Setting bed temp to {}C and waiting!'.format(temp))

            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                    read = self.handle.readline()
                    time.sleep(0.06)

    # M2PCS SPECIFIC FUNCTIONS
    def allon(self):
        """
        Turns pneumatic CHANNEL 1, 2, 3 ON
        """
        if self.prinout == 1:
            if self.verbose: print('Turning all channels on')
            self.handle.write(str.encode('M3\n'))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)
            self.handle.write(str.encode('M5\n'))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)
            self.handle.write(str.encode('M7\n'))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)
        elif self.printout == 0:
            self.channel_status = np.array([1,1,1])

    def alloff(self):
        """
        Turns pneumatic CHANNEL 1, 2, 3 OFF
        """
        if self.printout == 1:
            if self.verbose: print('Turning all channels off')
            self.handle.write(str.encode('M4\n'))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)
            self.handle.write(str.encode('M6\n'))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)
            self.handle.write(str.encode('M8\n'))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)
        elif self.printout == 0:
            self.channel_status = np.array([0,0,0])

    # M3/M5/M7
    def on(self, channel):
        """
        Turns pneumatic CHANNEL ON
        """
        if self.printout == 1:
            if self.verbose: print('Turning on channel {}'.format(channel))
            schannel = 'M{}\n'.format(channel*2 + 1)
            self.handle.write(str.encode(schannel))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)
        elif self.printout == 0:
            self.channel_status[channel - 1] = 1

    # M4/M6/M8
    def off(self, channel):
        """
        Turns pneumatic CHANNEL OFF
        """
        if self.printout == 1:
            if self.verbose: print('Turning off channel {}'.format(channel))
            schannel = 'M{}\n'.format(channel*2 + 2)
            self.handle.write(str.encode(schannel))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)
        elif self.printout == 0:
            self.channel_status[channel - 1] = 0

    def set_channel_delay(self, delay = 50):
        """
        Sets the delay time (in ms) between a channel turning on and the execution of another command. Can be used to fine tune under extrusion effects, depending on ink viscosity.
        """
        if self.printout == 1:
            if self.verbose: print('Setting channel delay to {} ms'.format(delay))
            self.handle.write(str.encode('M50 S{}\n'.format(delay)))
            read = self.handle.readline()
            while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                read = self.handle.readline()
                time.sleep(0.06)

    def set_tool_coords(self, tool = 1, x = 0, y = 0, z = 0):
        """
        Sets internally stored coordinates of each tool, used in switching commands
        """
        if self.verbose: print('Setting coordinates of tool {} to ({},{},{})'.format(tool, x, y, z))
        self.tool_coords[tool - 1] = [x, y, z]


    def change_tool(self, change_to = 1):
        """
        This subroutine automatically turns off all channels, and performs a predetermined z translation of z = change_height, and then moves (x,y) = (dx, dy) to allow for change between multiple nozzles. It also automatically lowers back to the z height it was at previously, continuing printing after switching active tools
        """
        if self.printout == 1:
            old_tool = self.current_tool
            if self.verbose: print('Changing from tool {} to tool {}'.format(old_tool, change_to))
            self.alloff()
            old_coord_sys = self.current_coord_sys
            old_coords = self.coords
            self.coord_sys(coord_sys = 'rel')
            coord_change = self.tool_coords[change_to - 1] - self.tool_coords[self.current_tool - 1]
            self.move(x = coord_change[0], y = coord_change[1], z = coord_change[2])
            self.current_tool = change_to
            self.coord_sys(coord_sys = old_coord_sys)
            self.set_current_coords(x = old_coords[0], y = old_coords[1], z = old_coords[2])

    def path_vis(self, zrange):
        """
        Takes the (x, y, z) coordinates generated from mp.mopen(printout = 0), and plots them into a 3D line graph to check a print path before actually sending commands to the Makergear. Visualization function will use whatever coordinate system you explicity designate using coord. If coord isn't explicitly called, the coordinate system used by the visualization tool will be absolute. When using path_vis, the file directory of the path coordinates needs to be explicity set, unlike when it is implictly called inside mclose.
        """
        if self.verbose: print('Generating path visualization')
        coord_array = np.zeros([1, 3])
        channel_array = np.zeros([1, 3])

        if self.current_coord_sys == 'abs':
            with open(self.fid, "r") as data:
                for line in data:
                    raw = line.split(' ')
                    coords = [float(raw[0]), float(raw[1]), float(raw[2])]
                    channels = [float(raw[3]), float(raw[4]), float(raw[5])]
                    coord_array = np.append(coord_array, [coords[0], coords[1], coords[2]])
                    channel_array = np.append(channel_array, [channels[0], channels[1], channels[2]])
            coord_array.shape = (int(len(coord_array)/3), 3)

        elif self.current_coord_sys == 'rel':
            old_coords = [0,0,0]
            with open(self.fid, "r") as data:
                for line in data:
                    raw = line.split(' ')
                    coords = [float(raw[0]), float(raw[1]), float(raw[2])]
                    channels = [float(raw[3]), float(raw[4]), float(raw[5])]
                    coord_array = np.append(coord_array, [coords[0] + old_coords[0], coords[1] + old_coords[1], coords[2] + old_coords[2]])
                    old_coords =  [coords[0] + old_coords[0], coords[1] + old_coords[1], coords[2] + old_coords[2]]
                    channel_array = np.append(channel_array, [channels[0], channels[1], channels[2]])
            coord_array.shape = (int(len(coord_array)/3), 3)

        channel_array.shape = (int(len(channel_array)/3), 3)
        num_moves = channel_array.shape[0]

        ch_split_index = np.array([])
        for i in range(num_moves-1):
            if np.array_equal(channel_array[i,:], channel_array[i+1,:]) == False:
                ch_split_index = np.append(ch_split_index, i+1)

        x_coord = coord_array[:,0]
        y_coord = coord_array[:,1]
        z_coord = coord_array[:,2]
        ch_split_index = ch_split_index.astype(int)

        start_pt = [x_coord[0], y_coord[0], z_coord[0]]
        end_pt = [x_coord[-1], y_coord[-1], z_coord[-1]]

        x_split = np.split(x_coord, ch_split_index)
        y_split = np.split(y_coord, ch_split_index)
        z_split = np.split(z_coord, ch_split_index)
        ch_split = np.split(channel_array, ch_split_index)
        num_lines = len(x_split)

        for j in range(num_lines - 1):
            x_split[j+1] = np.insert(x_split[j+1], 0, x_split[j][-1])
            y_split[j+1] = np.insert(y_split[j+1], 0, y_split[j][-1])
            z_split[j+1] = np.insert(z_split[j+1], 0, z_split[j][-1])

        xmin = np.min(x_coord)
        xmax = np.max(x_coord)
        ymin = np.min(y_coord)
        ymax = np.max(y_coord)
        zmax = np.max(z_coord)

        xymax = (xmax>=ymax)*xmax + (ymax>xmax)*ymax
        xymin = (xmin<=ymin)*ymin + (ymin<xmin)*ymin

        fig = plt.figure()
        ax = fig.gca(projection=Axes3D.name)

        ax.set_xlim3d(xymin, xymax)
        ax.set_ylim3d(xymin, xymax)

        if zrange == 'fit':
            ax.set_zlim3d(0, zmax)
        else:
            ax.set_zlim3d(zrange[0], zrange[1])

        ax.set_xlabel('X axis [mm]')
        ax.set_ylabel('Y axis [mm]')
        ax.set_zlabel('Z axis [mm]')

        for k in range(num_lines):

            if np.array_equal(ch_split[k][0], [0, 0, 0]):
                cstr = '#484848'
                linestyle = ':'
            elif np.array_equal(ch_split[k][0], [1, 0, 0]):
                cstr = '#0000ff'
                linestyle = '-'
            elif np.array_equal(ch_split[k][0], [0, 1, 0]):
                cstr = '#00ff00'
                linestyle = '-'
            elif np.array_equal(ch_split[k][0], [0, 0, 1]):
                cstr = '#ff0000'
                linestyle = '-'
            ax.plot(x_split[k], y_split[k], z_split[k], color = cstr, linewidth = 2, linestyle = linestyle)

        ch1_patch = mpatches.Patch(color='#0000ff', label='Channel 1')
        ch2_patch = mpatches.Patch(color='#00ff00', label='Channel 2')
        ch3_patch = mpatches.Patch(color='#ff0000', label='Channel 3')
        ax.scatter(start_pt[0], start_pt[1], start_pt[2], c='#7E7C66', marker='o')
        ax.scatter(end_pt[0], end_pt[1], end_pt[2], c='k', marker='o')
        plt.legend(handles=[ch1_patch, ch2_patch, ch3_patch], loc = 'best')
        plt.title('M2PCS Print Path Visualization')
        plt.show()


#Additional functions outside of the M2 CLASS
def prompt(com, baud):
    """
    Allows for quick, native GCode serial communication with the M2, provided that the proper com port and baud rate are selected, and match what is found in system settings. To exit the command prompt environment, just type exit in the IPython console.
    """
    escape = 0
    cmd = ''
    #Create and define serial port parameters
    handle = serial.Serial(com, baud, timeout = 0)
    time.sleep(2) # Make sure to give it enough time to initialize
    print("Enter a GCode command. To exit, type \'exit\'")
    handle.write(str.encode('M17\n'))
    x = y = z = 0
    xval = yval = zval = 0
    while escape == 0:
        cmd = input('>> ')
        if cmd == 'exit':
            handle.close()
            print("Serial port disconnected")
            escape = 1
        else:
            handle.write(str.encode('{}\n'.format(cmd)))
            if cmd =='G28':
                x = y = z = 0
                xval = yval = zval = 0
            cmd_split = cmd.split(' ')
            for a in cmd_split:
                xchar = a.split('X')
                ychar = a.split('Y')
                zchar = a.split('Z')
                if xchar[0] == '' and len(xchar) > 1 and xchar[1] != '':
                    xval = float(xchar[1])
                    x = x + xval
                if ychar[0] == '' and len(ychar) > 1 and ychar[1] != '':
                    yval = float(ychar[1])
                    y = y + yval
                if zchar[0] == '' and len(zchar) > 1 and zchar[1] != '':
                    zval = float(zchar[1])
                    z = z + zval
            print('Currently at ({}, {}, {})'.format(x, y, z))

def file_read(fid, com, baud, dx, dy):
    """
    Reads in a text file of GCode line by line, and waits for the M2 to acknowledge that it received the command before sending another, maintaining print accuracy.
    """
    #Create and define serial port parameters
    handle = serial.Serial(com, baud, timeout = 100)
    time.sleep(2) # Make sure to give it enough time to initialize

    print('Serial port initialized')
    for _ in range(21): # Reads in all 21 lines of initalization text for the M2
        handle.readline()
    print('Beginning print')
    with open(fid, "r") as gcode:
        for line in gcode:
            split_line = line.split(';') # chops off comments
            if split_line[0] != '':      # makes sure it's not a comment-only line of GCode
                handle.write(str.encode('{}\n'.format(split_line[0]))) #Reads in text file (GCode) line by line and sends commands to M2
                print(split_line[0])
                read = handle.readline()
                if split_line[0][0] != 'M':
                    while read[0:2] != b'ok': #Waits for printer to send 'ok' command before sending the next command, ensuring print accuracy
                        read = handle.readline()

    time.sleep(2)
    print('Print complete!\nSerial port closed')
    handle.close() #Closes serial port 


In [ ]:
#[M2PCS] Movement Tool
from tkinter import Tk, Label, Button, Entry, DoubleVar, StringVar
from tkinter.ttk import Combobox
import serial.tools.list_ports


window = Tk()
window.title('[M2PCS] Movement Tool')
window.geometry('660x415')

xcoord = DoubleVar()
ycoord = DoubleVar()
zcoord = DoubleVar()
rspeed = StringVar()
rspeed.set('0')

lbl_xy = Label(text = 'X/Y', font = ('Courier New',12))
lbl_xy.grid(column = 1, row = 1)

btn_yp = Button(text = '↑', font = ('Courier New',18), bg = 'light blue', state = 'disabled')
btn_yp.grid(column = 1, row = 0, pady = (20, 0))

btn_ym = Button(text = '↓', font = ('Courier New',18), bg = 'light blue', state = 'disabled')
btn_ym.grid(column = 1, row = 2)

btn_xp = Button(text = '→', font = ('Courier New',18), bg = 'light blue', state = 'disabled')
btn_xp.grid(column = 2, row = 1)

btn_xm = Button(text = '←', font = ('Courier New',18), bg = 'light blue', state = 'disabled')
btn_xm.grid(column = 0, row = 1, padx = (20, 0))

lbl_z = Label(text = 'Z', font = ('Courier New',12))
lbl_z.grid(column = 0, row = 6, sticky = 'E', rowspan = 2)

btn_zp = Button(text = '↑', font = ('Courier New',18), bg = 'light green', state = 'disabled')
btn_zp.grid(column = 1, row = 6)

btn_zm = Button(text = '↓', font = ('Courier New',18), bg = 'light green', state = 'disabled')
btn_zm.grid(column = 1, row = 7)

lbl_x = Label(text = 'X:', font = ('Courier New',12))
lbl_x.grid(column = 3, row = 3)
entry_x = Entry(fg = 'red', width = 6)
entry_x.insert(0, '***')
entry_x.grid(column = 4, row = 3)
lbl_xpm = Label(text = '±:', font = ('Courier New',12))
lbl_xpm.grid(column = 5, row = 3)
combo_xpm = Combobox(width = 6, values = (0.1, 0.5, 1, 5, 10, 20, 50, 100), state = 'disabled')
combo_xpm.grid(column = 6, row = 3)
combo_xpm.current(3)

lbl_y = Label(text = 'Y:', font = ('Courier New',12))
lbl_y.grid(column = 3, row = 4)
entry_y = Entry(fg = 'red', width = 6)
entry_y.insert(0, '***')
entry_y.grid(column = 4, row = 4)
lbl_ypm = Label(text = '±:', font = ('Courier New',12))
lbl_ypm.grid(column = 5, row = 4)
combo_ypm = Combobox(width = 6, values = (0.1, 0.5, 1, 5, 10, 20, 50, 100), state = 'disabled')
combo_ypm.grid(column = 6, row = 4)
combo_ypm.current(3)

lbl_z = Label(text = 'Z:', font = ('Courier New',12))
lbl_z.grid(column = 3, row = 5)
entry_z = Entry(fg = 'red', width = 6)
entry_z.insert(0, '***')
entry_z.grid(column = 4, row = 5)
lbl_zpm = Label(text = '±:', font = ('Courier New',12))
lbl_zpm.grid(column = 5, row = 5)
combo_zpm = Combobox(width = 6, values = (0.1, 0.5, 1, 5, 10, 20, 50, 100), state = 'disabled')
combo_zpm.grid(column = 6, row = 5)
combo_zpm.current(3)

#Channel Control Buttons
btn_ch1 = Button(text = 'CH 1', font = ('Courier New',12), state = 'disabled', relief = 'raised', bg = 'lightgray')
btn_ch1.grid(row = 6, column = 7)

btn_ch2 = Button(text = 'CH 2', font = ('Courier New',12), state = 'disabled', relief = 'raised', bg = 'lightgray')
btn_ch2.grid(row = 6, column = 8)

btn_ch3 = Button(text = 'CH 3', font = ('Courier New',12), state = 'disabled', relief = 'raised', bg = 'lightgray')
btn_ch3.grid(row = 6, column = 9)

btn_allon = Button(text = 'ALL ON', font = ('Courier New',12), state = 'disabled', relief = 'raised', bg = 'lightgray')
btn_allon.grid(row = 5, column = 8)

btn_alloff = Button(text = 'ALL OFF', font = ('Courier New',12), state = 'disabled', relief = 'raised', bg = 'lightgray')
btn_alloff.grid(row = 7, column = 8)


#Homing Buttons
btn_home_all = Button(text = 'Home All', font = ('Courier New',12), state = 'disabled', bg = 'lightgray')
btn_home_all.grid(column = 7, row = 0, columnspan = 3)

btn_home_x = Button(text = 'Home X', font = ('Courier New',12), state = 'disabled', bg = 'lightgray')
btn_home_x.grid(column = 7, row = 1)

btn_home_y = Button(text = 'Home Y', font = ('Courier New',12), state = 'disabled', bg = 'lightgray')
btn_home_y.grid(column = 8, row = 1)

btn_home_z = Button(text = 'Home Z', font = ('Courier New',12), state = 'disabled', bg = 'lightgray')
btn_home_z.grid(column = 9, row = 1)

#Rotate Buttons
btn_rotate_p = Button(text = 'ROTATE +', font = ('Courier New',10), state = 'disabled', bg = 'lightgray')
btn_rotate_p.grid(column = 9, row = 2)

entry_rotate = Entry(fg = 'black', width = 6, textvariable = rspeed)
entry_rotate.grid(column = 8, row = 2)

btn_rotate_n = Button(text = 'ROTATE -', font = ('Courier New',10), state = 'disabled', bg = 'lightgray')
btn_rotate_n.grid(column = 7, row = 2)

btn_rotate_set = Button(text = 'SET SPEED', font = ('Courier New',10), state = 'disabled', bg = 'lightgray')
btn_rotate_set.grid(column = 8, row = 3)

#COMPORT
lbl_com = Label(window, text = 'COMPORT: ****',font = ("Courier New", 8), fg = 'red')
lbl_com.grid(column = 0, row = 9, sticky = 'W', columnspan = 3, pady = (20, 0))
comm_data = serial.tools.list_ports.comports()
comm_list = []
for i in range(len(comm_data)):
    comm_list.append(comm_data[i].device)
comm_tuple = tuple(comm_list)
combo_com = Combobox(window, font = ("Courier New", 8))
combo_com.grid(column = 3, row = 9,columnspan = 3, padx = (10, 0))
combo_com['values'] = comm_tuple
btn_com_set = Button(window, text = 'SET', font = ("Courier New", 8), bg = 'lightgray')
btn_com_set.grid(column = 6, row = 9)

#SPEED
lbl_speed = Label(window, text = 'BAUDRATE: ****** BPS',font = ("Courier New", 8), fg = 'red')
lbl_speed.grid(column = 0, row = 10, sticky = 'W', columnspan = 3)
combo_speed = Combobox(window, font = ("Courier New", 8))
combo_speed.grid(column = 3, row = 10, columnspan = 3, padx = (10, 0))
combo_speed['values'] = (9600, 14400, 19200, 38400, 57600, 115200, 128000, 256000)
combo_speed.current(5)
btn_speed_set = Button(window, text = 'SET', font = ("Courier New", 8), bg = 'lightgray')
btn_speed_set.grid(column = 6, row = 10)

btn_connect = Button(text = 'Connect', bg = 'lightgreen', font = ("Courier New", 20), state = 'disabled')
btn_connect.grid(column = 7, row = 9, columnspan = 3, rowspan = 2)

#FUNCTION DEFINITIONS

def set_com():
    global connect_status
    global comport
    comport = combo_com.get()
    lbl_com.configure(text = 'COMPORT: {}'.format(comport), fg = 'blue')
    btn_connect.configure(text = 'Connect', bg = 'lightgreen', font = ("Courier New", 20), command = lambda: connect(), state = 'normal')
btn_com_set.config(command = set_com)

#SET SPEED
def set_speed():
    global speed
    speed = combo_speed.get()
    lbl_speed.configure(text = 'BAUDRATE: {} BPS'.format(speed), fg = 'blue')    
btn_speed_set.config(command = set_speed)

#SET Rotate
def set_rotate():
    global rotate
    global mg
    rotate = int(entry_rotate.get())
    rotate(mg, speed = rotate)
btn_rotate_set.config(command = set_rotate)

#UPDATE Rotate
def update_rotate(direction = 'up'):
    global rotate
    if direction == 'up':
        rotate = int(entry_rotate.get()) + 1
    elif direction == 'down':
        rotate = int(entry_rotate.get()) - 1
    rspeed.set(str(rotate))
btn_rotate_p.config(command = lambda: update_rotate(direction = 'up'))
btn_rotate_n.config(command = lambda: update_rotate(direction = 'down'))

def connect():
    global connect_status
    global mg
    global comport
    global speed
    mg = mopen(comport, speed)
    coord(mg, coord = 'rel')
    delay_set(mg, delay = 1)
    connect_status = 1
    btn_connect.configure(text = 'Disconnect', bg = 'salmon', font = ("Courier New", 20), command = lambda: disconnect())
    btn_xp.configure(state = 'normal')
    btn_xm.configure(state = 'normal')
    btn_yp.configure(state = 'normal')
    btn_ym.configure(state = 'normal')
    btn_zp.configure(state = 'normal')
    btn_zm.configure(state = 'normal')
    btn_rotate_p.configure(state = 'normal')
    btn_rotate_n.configure(state = 'normal')
    btn_rotate_set.configure(state = 'normal')
    btn_home_all.configure(state = 'normal')
    btn_home_x.configure(state = 'normal')
    btn_home_y.configure(state = 'normal')
    btn_home_z.configure(state = 'normal')
    combo_xpm.configure(state = 'normal')
    combo_ypm.configure(state = 'normal')
    combo_zpm.configure(state = 'normal')
    btn_ch1.configure(state = 'normal')
    btn_ch2.configure(state = 'normal')
    btn_ch3.configure(state = 'normal')
    btn_allon.configure(state = 'normal')
    btn_alloff.configure(state = 'normal')
    
def disconnect():
    global connect_status
    global mg
    mclose(mg)
    connect_status = 0
    btn_connect.configure(text = 'Connect', bg = 'lightgreen', font = ("Courier New", 20), command = lambda: connect())

def home(axes = ''):
    global mg
    home(mg, axes = axes)
    
    if axes == 'X Y Z':
        xcoord.set(0.0)
        ycoord.set(0.0)
        zcoord.set(0.0)
    elif axes == 'X':
        xcoord.set(0.0)
    elif axes == 'Y':
        ycoord.set(0.0)
    elif axes == 'Z':
        zcoord.set(0.0)
    
    entry_x.configure(textvariable = xcoord, fg = 'black')
    entry_y.configure(textvariable = ycoord, fg = 'black')
    entry_z.configure(textvariable = zcoord, fg = 'black')
        
btn_home_all.configure(command = lambda: home('X Y Z'))
btn_home_x.configure(command = lambda: home('X'))
btn_home_y.configure(command = lambda: home('Y'))
btn_home_z.configure(command = lambda: home('Z'))

def update_coord(dx = 0, dy = 0, dz = 0):
    global mg
    move(mg, x = dx, y = dy, z = dz)
    xcoord.set(round(xcoord.get() + float(dx), 1))
    ycoord.set(round(ycoord.get() + float(dy), 1))
    zcoord.set(round(zcoord.get() + float(dz), 1))
    entry_x.configure(textvariable = xcoord, fg = 'black')
    entry_y.configure(textvariable = ycoord, fg = 'black')
    entry_z.configure(textvariable = zcoord, fg = 'black')
    
btn_xp.configure(command = lambda: update_coord(dx = combo_xpm.get()))
btn_xm.configure(command = lambda: update_coord(dx = -float(combo_xpm.get())))
btn_yp.configure(command = lambda: update_coord(dy = combo_ypm.get()))
btn_ym.configure(command = lambda: update_coord(dy = -float(combo_ypm.get())))
btn_zp.configure(command = lambda: update_coord(dz = -float(combo_zpm.get())))
btn_zm.configure(command = lambda: update_coord(dz = combo_zpm.get()))

def channel_on(channels = 0):
    global mg
    if channels == 1:
        ch1on(mg)
        btn_ch1.configure(relief = 'sunken', command = lambda: channel_off(channels = 1))
    elif channels == 2:
        ch2on(mg)
        btn_ch2.configure(relief = 'sunken', command = lambda: channel_off(channels = 2))
    elif channels == 3:
        ch3on(mg)
        btn_ch3.configure(relief = 'sunken', command = lambda: channel_off(channels = 3))
    elif channels == 4:
        allon(mg)
        btn_ch1.configure(relief = 'sunken', command = lambda: channel_off(channels = 1))
        btn_ch2.configure(relief = 'sunken', command = lambda: channel_off(channels = 2))
        btn_ch3.configure(relief = 'sunken', command = lambda: channel_off(channels = 3))
        
def channel_off(channels = 0):
    global mg
    if channels == 1:
        ch1off(mg)
        btn_ch1.configure(relief = 'raised', command = lambda: channel_on(channels = 1))
    elif channels == 2:
        ch2off(mg)
        btn_ch2.configure(relief = 'raised', command = lambda: channel_on(channels = 2))
    elif channels == 3:
        ch3off(mg)
        btn_ch3.configure(relief = 'raised', command = lambda: channel_on(channels = 3))
    elif channels == 4:
        alloff(mg)
        btn_ch1.configure(relief = 'raised', command = lambda: channel_on(channels = 1))
        btn_ch2.configure(relief = 'raised', command = lambda: channel_on(channels = 2))
        btn_ch3.configure(relief = 'raised', command = lambda: channel_on(channels = 3))

btn_ch1.configure(command = lambda: channel_on(channels = 1))
btn_ch2.configure(command = lambda: channel_on(channels = 2))
btn_ch3.configure(command = lambda: channel_on(channels = 3))
btn_allon.configure(command = lambda: channel_on(channels = 4))
btn_alloff.configure(command = lambda: channel_off(channels = 4))
        
window.mainloop()

Serial port connected


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\littl\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\littl\AppData\Local\Temp\ipykernel_32004\1960390763.py", line 151, in <lambda>
    btn_connect.configure(text = 'Connect', bg = 'lightgreen', font = ("Courier New", 20), command = lambda: connect(), state = 'normal')
  File "C:\Users\littl\AppData\Local\Temp\ipykernel_32004\1960390763.py", line 186, in connect
    coord(mg, coord = 'rel')
  File "C:\Users\littl\AppData\Local\Temp\ipykernel_32004\1685227954.py", line 306, in coord
    read = handle.readline()
  File "C:\Users\littl\anaconda3\lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))


Serial port connected
Set to relative coordinates
Channel ON delay time changed to 1 ms
